# pydantic

以下のようなyamlファイルがあったとする  
また、各属性は何かしらの制限がある(nameは文字列で空白不可、レストランの席数は負ではないなど)  
```
name: Viafore's
owner: Pat Viafore
address: 123 Fake St. Fakington, FA 01234
employees:
  - name: Pat Viafore
    position: Chef
    payment_details:
      bank_details:
        routing_number: "123456789"
        account_number: "123456789012"
  - name: Made-up McGee
    position: Server
    payment_details:
      bank_details:
        routing_number: "123456789"
        account_number: "123456789012"
  - name: Fabricated Frank
    position: Sous Chef
    payment_details:
      bank_details:
        routing_number: "123456789"
        account_number: "123456789012"
  - name: Illusory Ilsa
    position: Host
    payment_details:
      bank_details:
        routing_number: "123456789"
        account_number: "123456789012"
dishes:
  - name: Pasta And Sausage
    price_in_cents: 1295
    description: Rigatoni and Sausage with a Tomato-Garlic-Basil Sauce
  - name: Pasta Bolognese
    price_in_cents: 1495
    description: Spaghetti with a rich Tomato and Beef Sauce
  - name: Caprese Salad
    price_in_cents: 795
    description: Tomato, Buffalo Mozzarella, and Basil
    picture: caprese.png
number_o_seats: 12
to_go: true
delivery: false

```

## TypeDicts
欠損フィールドやデータ型が間違っていないかをチェックできる

In [1]:
from typing import Literal,TypedDict,Union
class AccountAndRoutingNumber(TypedDict):
    account_number: str
    routing_number: str

class BankDetails(TypedDict):
    bank_details: AccountAndRoutingNumber

class Address(TypedDict):
    address: str

AddressOrBankDetails = Union[Address, BankDetails]

Position = Literal['Chef', 'Sous Chef', 'Host',
                   'Server', 'Delivery Driver']

class Dish(TypedDict):
    name: str
    price_in_cents: int
    description: str

class DishWithOptionalPicture(Dish, TypedDict, total=False):
    picture: str

class Employee(TypedDict):
    name: str
    position: Position
    payment_details: AddressOrBankDetails

class Restaurant(TypedDict):
    name: str
    owner: str
    address: str
    employees: list[Employee]
    dishes: list[Dish]
    number_of_seats: int
    to_go: bool
    delivery: bool


In [2]:
# yamlファイルを読み込む関数
import yaml
def load_restaurant(filename: str) -> Restaurant:
    with open(filename) as yaml_file:
        return yaml.safe_load(yaml_file)

In [3]:
load_restaurant(filename="../document/restaurant.yaml")

{'name': "Viafore's",
 'owner': 'Pat Viafore',
 'address': '123 Fake St. Fakington, FA 01234',
 'employees': [{'name': 'Pat Viafore',
   'position': 'Chef',
   'payment_details': {'bank_details': {'routing_number': '123456789',
     'account_number': '123456789012'}}},
  {'name': 'Made-up McGee',
   'position': 'Server',
   'payment_details': {'bank_details': {'routing_number': '123456789',
     'account_number': '123456789012'}}},
  {'name': 'Fabricated Frank',
   'position': 'Sous Chef',
   'payment_details': {'bank_details': {'routing_number': '123456789',
     'account_number': '123456789012'}}},
  {'name': 'Illusory Ilsa',
   'position': 'Host',
   'payment_details': {'bank_details': {'routing_number': '123456789',
     'account_number': '123456789012'}}}],
 'dishes': [{'name': 'Pasta And Sausage',
   'price_in_cents': 1295,
   'description': 'Rigatoni and Sausage with a Tomato-Garlic-Basil Sauce'},
  {'name': 'Pasta Bolognese',
   'price_in_cents': 1495,
   'description': 'Spaghe

## pydantic

In [4]:
from pydantic import ValidationError
from pydantic.dataclasses import dataclass
from typing import Literal, List, Optional, TypedDict, Union

@dataclass
class AccountAndRoutingNumber():
    account_number: str
    routing_number: str

@dataclass
class BankDetails:
    bank_details: AccountAndRoutingNumber

@dataclass
class Address:
    address: str

AddressOrBankDetails = Union[Address, BankDetails]

Position = Literal['Chef', 'Sous Chef', 'Host',
                   'Server', 'Delivery Driver']

@dataclass
class Dish:
    name: str
    price_in_cents: int
    description: str
    picture: Optional[str] = None

@dataclass
class Employee:
    name: str
    position: Position
    payment_details: AddressOrBankDetails

@dataclass
class Restaurant:
    name: str
    owner: str
    address: str
    employees: list[Employee]
    dishes: list[Dish]
    number_of_seats: int
    to_go: bool
    delivery: bool

In [5]:
# 読み取りの関数は以下の内容に変更される
def load_restaurant(filename: str) -> Restaurant:
    with open(filename) as yaml_file:
        data = yaml.safe_load(yaml_file)
        return Restaurant(**data)

load_restaurant(filename="../document/restaurant.yaml")

Restaurant(name="Viafore's", owner='Pat Viafore', address='123 Fake St. Fakington, FA 01234', employees=[Employee(name='Pat Viafore', position='Chef', payment_details=BankDetails(bank_details=AccountAndRoutingNumber(account_number='123456789012', routing_number='123456789'))), Employee(name='Made-up McGee', position='Server', payment_details=BankDetails(bank_details=AccountAndRoutingNumber(account_number='123456789012', routing_number='123456789'))), Employee(name='Fabricated Frank', position='Sous Chef', payment_details=BankDetails(bank_details=AccountAndRoutingNumber(account_number='123456789012', routing_number='123456789'))), Employee(name='Illusory Ilsa', position='Host', payment_details=BankDetails(bank_details=AccountAndRoutingNumber(account_number='123456789012', routing_number='123456789')))], dishes=[Dish(name='Pasta And Sausage', price_in_cents=1295, description='Rigatoni and Sausage with a Tomato-Garlic-Basil Sauce', picture=None), Dish(name='Pasta Bolognese', price_in_cent

In [8]:
# 誤ったデータを渡すとエラーが発生する -> 型が異なる, 必要なデータが存在しない

load_restaurant(filename="../document/restaurant_miss.yaml")

ValidationError: 1 validation error for Restaurant
employees.0.position
  Input should be 'Chef', 'Sous Chef', 'Host', 'Server' or 'Delivery Driver' [type=literal_error, input_value='SE', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/literal_error

### バリデータ

In [15]:
from pydantic.dataclasses import dataclass
from pydantic import constr, PositiveInt, ValidationError
from typing import Literal, Optional, Union

@dataclass
class AccountAndRoutingNumber:
    account_number: constr(min_length=9,max_length=9)
    routing_number: constr(min_length=8,max_length=12)


@dataclass
class BankDetails:
    bank_details: AccountAndRoutingNumber


@dataclass
class Address:
    address: constr(min_length=1)

AddressOrBankDetails = Union[Address, BankDetails]

Position = Literal['Chef', 'Sous Chef', 'Host',
                   'Server', 'Delivery Driver']
@dataclass
class Employee:
    name: str
    position: Position

@dataclass
class Dish:
    name: constr(min_length=1, max_length=16)
    price_in_cents: PositiveInt
    description: constr(min_length=1, max_length=80)
    picture: Optional[str] = None


@dataclass
class Dish:
    name: constr(min_length=1, max_length=16)

from pydantic import conlist,constr
from pydantic import validator
@dataclass
class Restaurant:
    name: constr(strict=r'^[a-zA-Z0-9 ]*$',
                   min_length=1, max_length=16)
    owner: constr(min_length=1)
    address: constr(min_length=1)
    employees: conlist(Employee, min_items=2)
    dishes: conlist(Dish, min_items=3)
    number_of_seats: PositiveInt
    to_go: bool
    delivery: bool

    @validator('employees')
    def check_chef_and_server(cls, employees):
        if (any(e for e in employees if e.position == 'Chef') and 
            any(e for e in employees if e.position == 'Server')):
                return employees
        raise ValueError('Must have at least one chef and one server')

TypeError: conlist() got an unexpected keyword argument 'min_items'

In [11]:
try:
    restaurant = Restaurant(**{
        'name': 'Dine n Dash',
        'owner': 'Pat Viafore',
        'address': '123 Fake St.',
        'employees': [Employee('Pat', 'Chef'), Employee('Joe', 'Chef')],
        'dishes': [Dish('abc'), Dish('def'), Dish('ghi')],
        'number_of_seats': 5,
        'to_go': False,
        'delivery': True
    })
    assert False, "Should not have been able to construct Restaurant"
except ValidationError:
    pass

### パースライブラリであるpydantic

In [19]:
from pydantic import dataclass

@dataclass
class Model:
    value: int

ImportError: cannot import name 'dataclass' from 'pydantic' (/usr/local/lib/python3.9/site-packages/pydantic/__init__.py)

In [17]:
Model(value=5.5)

ValidationError: 1 validation error for Model
value
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=5.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.4/v/int_from_float